In [12]:
import torch
from config import get_config
config = get_config()
# print(config)
for k,v in config.items():
    print(k,v)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
device = torch.device(device)

batch_size 8
val_epochs 20
lr 0.0001
src_vocab_size 25
tgt_vocab_size 20
seq_len 1024
d_model 960
model_folder weights
model_basename tmodel_
preload latest
experiment_name runs/tmodel
Using device: cuda


In [13]:
from dataloader import get_ds
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)


In [14]:
encoded = tokenizer_tgt.encode("54-122_167-589 | 1.10.640.10")
# Encoding(num_tokens=28, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
encoded.ids
for i in encoded.ids:
    print(tokenizer_tgt.decode([i]))


5
4
-
1
2
2
_
1
6
7
-
5
8
9
 
|
 
1
.
1
0
.
6
4
0
.
1
0


In [15]:
batch = next(iter(train_dataloader))
# print(batch)
print(batch.keys())
# print(type(batch))
encoder_input = batch['encoder_input'].to(device) # (B, seq_len)
print("encoder_input.shape", encoder_input.shape)
decoder_input = batch['decoder_input'].to(device) # (B, seq_len)
print("decoder_input.shape", decoder_input.shape)
encoder_mask = batch['encoder_mask'].to(device) # (B, 1, 1, seq_len)
print("encoder_mask.shape", encoder_mask.shape)
decoder_mask = batch['decoder_mask'].to(device) # (B, 1, seq_len, seq_len)
print("decoder_mask.shape", decoder_mask.shape)
label = batch['label'].to(device) # (B, seq_len)
print("label.shape", label.shape)
source = batch['src_text']
print(batch['src_text'])
print(batch['tgt_text'])
# print(decoder_mask)

dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'src_text', 'tgt_text'])
encoder_input.shape torch.Size([8, 1024])
decoder_input.shape torch.Size([8, 1024])
encoder_mask.shape torch.Size([8, 1, 1, 1024])
decoder_mask.shape torch.Size([8, 1, 1024, 1024])
label.shape torch.Size([8, 1024])
['MGRYSREPDNPAKSCKARGSNLRVHFKNTYETAMAIKKLPLRRAVRYLKNVVDKKECIPFRRFNGGVGRCAQAKQFGTTQGRWPKKSAEF', 'ELEPVEPYSNEDLDWTDESSRVSREYENEDERDVELVSTEVDDWESYDTVFIGYPIWWAIAAWPVNDFVEANDFTGKTVIPFCTSATSGMGESGELLAELAGTGDWQEGQRFRSGADDKEIQEWVESLGL', 'MEGNTEKPFGIVRGITFSTMNMKRTNLAETEGNPGDQISTLIFRNVEWMGYSETLKISIAMLHLKR', 'MIAGTGIDIASISRMEKGLSRFGARFSRRLLSPGEETLFPAGTKRAAAFLAGRWAAKEAAVKALGTGFAGGVAPSDVSVLSLPSGQPFIRFAGRARERAEELGVTSVHVSITHDAGFAAAIVLLETGLR', 'MRAVTWQGRRDVRVETVPDPRIEEPTDVIVRVTSTGICGSDLHLYEPLGPFLDPGDILGHEPMGIVEEVGGAVTALKPGDRVVVPFNVSCGDCFMCDQGLQSQCETTQVTEYGTGAALFGYTKLYGQVPGGQAEYLRVPFGNTLPVKVGHGPPDDRYVYLSDVLPTAWQAVEYASVPRDGTVVVLGLGPIGDMAARIALHRGAGRVLGVDLVPDRLNRAAAHGVIPLDWRRYGKDLPEAVA

In [21]:
from model import build_transformer_esmc
from config import latest_weights_file_path, get_weights_file_path
model = build_transformer_esmc(tokenizer_tgt.get_vocab_size(), config['seq_len'], d_model=config['d_model']).to(device)

preload = config['preload']
model_filename = latest_weights_file_path(config)
model_filename = "weights/tmodel_400.pt"
if model_filename:
    print(f'Preloading model {model_filename}')
    state = torch.load(model_filename)
    model.load_state_dict(state['model_state_dict'])
else:
    print('No model to preload, starting from scratch')

Preloading model weights/tmodel_400.pt


In [17]:
# Print the number of trainable parameters in the model and all the numbers
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_parameters(model)
print(f"Total trainable parameters: {total_params:,}")

# # Print all parameter counts by module
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(f"{name}: {param.numel():,}")


Total trainable parameters: 67,922,708


In [30]:
encoder_output = model.encode(batch['src_text'], encoder_mask) # (B, seq_len, d_model)
print(encoder_output.shape)
print(encoder_output)

torch.Size([1, 1024, 960])
tensor([[[ 0.0062, -0.0049,  0.0013,  ...,  0.0020, -0.0018, -0.0099],
         [-0.0284,  0.0368, -0.0025,  ...,  0.0178, -0.0490,  0.0502],
         [ 0.0160,  0.0379,  0.0275,  ...,  0.0014, -0.0507,  0.0165],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       device='cuda:0')


In [ ]:
from model import build_transformer
trans_model = build_transformer(tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size(), config['seq_len'], config['seq_len'], d_model=config['d_model']).to(device)
encoder_output = trans_model.encode(encoder_input, encoder_mask) # (B, seq_len, d_model)
print(encoder_output.shape)
print(encoder_output)

torch.Size([1, 1024, 960])
tensor([[[-1.0622, -0.9160, -1.9601,  ...,  0.6036, -1.5966, -0.4590],
         [-0.8933, -2.6225, -0.6266,  ...,  0.6246, -2.2994, -0.6068],
         [-0.2721, -2.5294, -0.6001,  ...,  2.1170, -0.8606,  0.4062],
         ...,
         [-0.7625, -2.8094, -1.2348,  ...,  0.9257, -0.8632, -0.4945],
         [-1.3344, -3.0120, -1.7427,  ...,  1.5122, -1.2151, -0.8662],
         [-1.0707, -2.5780, -1.5908,  ...,  0.8420, -1.5378, -0.5994]]],
       device='cuda:0', grad_fn=<AddBackward0>)


: 

In [23]:
from dataset import causal_mask

batch = next(iter(val_dataloader))
# print(batch)
print(batch.keys())
# print(type(batch))
encoder_input = batch['encoder_input'].to(device) # (B, seq_len)
print("encoder_input.shape", encoder_input.shape)
decoder_input = batch['decoder_input'].to(device) # (B, seq_len)
print("decoder_input.shape", decoder_input.shape)
encoder_mask = batch['encoder_mask'].to(device) # (B, 1, 1, seq_len)
print("encoder_mask.shape", encoder_mask.shape)
decoder_mask = batch['decoder_mask'].to(device) # (B, 1, seq_len, seq_len)
print("decoder_mask.shape", decoder_mask.shape)
label = batch['label'].to(device) # (B, seq_len)
print("label.shape", label.shape)
source = batch['src_text']
print(batch['src_text'])
print(batch['tgt_text'])



sos_idx = tokenizer_tgt.token_to_id('[SOS]')
eos_idx = tokenizer_tgt.token_to_id('[EOS]')

# Precompute the encoder output and reuse it for every step
encoder_output = model.encode(source, encoder_mask)
# Initialize the decoder input with the sos token
decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(encoder_input).to(device)
i = 0
while True:
    if decoder_input.size(1) == 30:
        break

    # build mask for target
    decoder_mask = causal_mask(decoder_input.size(1)).type_as(encoder_input).to(device)

    # calculate output
    out = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)

    # get next token
    prob = model.project(out[:, -1])
    _, next_word = torch.max(prob, dim=1)
    tgt_token = tokenizer_tgt.decode([next_word.item()])
    print(i,tgt_token)
    decoder_input = torch.cat(
        [decoder_input, torch.empty(1, 1).fill_(next_word.item()).type_as(encoder_input).to(device)], dim=1
    )

    if next_word == eos_idx:
        break
    i += 1

print(decoder_input.squeeze(0))
tokenizer_tgt.decode(decoder_input.squeeze(0).tolist())
# return decoder_input.squeeze(0)

dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'src_text', 'tgt_text'])
encoder_input.shape torch.Size([1, 1024])
decoder_input.shape torch.Size([1, 1024])
encoder_mask.shape torch.Size([1, 1, 1, 1024])
decoder_mask.shape torch.Size([1, 1, 1024, 1024])
label.shape torch.Size([1, 1024])
['MLINVLLTLWFYTAHGQGKPCSYPVIKHGRLYYSYRGYFPARVHQQFVYSCDHHFVPPSQRSWDHLTCTAEGWSPEEPCLRQCIFNYLENGHTPHREEKYLQGETVRVRCYEGYSLQNDQNTMTCTESGWSPPPRCIRVKTCSKSNIRIENGFLSESTFTYPLNKQTEYKCKPGYVTADGKTSGLITCLKNGWSAQPVCIKSCDRPVFEKARVKSDGTWFRLNDRLDYECVDGYENRDGRTTGSIVCGQDGWSDKAACYERECSIPEMDPYLNAYPRKETYKVGDVLKFSCSQGRIMVGADSVQCYHFGWSPKLPTCKVKSCALPPELPNGKRKEIHKEEYAHNEVVEYACNPKFLMKGSHKIQCVDGEWTALPVCIEEERTCGNIPDLDHGDVKPSVPPYHHGDSVEFSCREAFTMIGPRFITCISGEWTQPPQSKIQLCPPPPQVPNACDMTTTVNYQDGEKISILCKENYLIQDAEEIVCKGGRWQSIPRCIEKIGCSQPPQLDHGTINSSSSAEERREIHEQRLYAHGTKLSYTCEEGFEISENNVIICHMGKWSSPPQCVGLPCGLPPYIQNGVISHKKDSYQYGEEVTYDCDEGFGTDGPASIRCLGGEWSRPQDCISTNCVNLPTFEDAVLTDREKDFYRSGEKVAFKCLSYYQLDGSNTIQCIK

'1 - 1 1 3   |   3 . 4 0 . 1 0 . 1 0'

In [1]:
from model import build_transformer_esmc
from dataset import causal_mask
from config import get_config, get_weights_file_path
from dataloader import get_ds

import torch
import pandas as pd

# Define the device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
device = torch.device(device)

def greedy_decode(model, source, source_mask, tokenizer_tgt, max_len):
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    # Precompute the encoder output and reuse it for every step
    encoder_output = model.encode(source, source_mask)
    # Initialize the decoder input with the sos token
    decoder_input = torch.empty(1, 1).fill_(sos_idx).type(torch.int64).to(device)
    while True:
        if decoder_input.size(1) == max_len:
            break

        # build mask for target
        decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)

        # calculate output
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

        # get next token
        prob = model.project(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat(
            [decoder_input, torch.empty(1, 1).fill_(next_word.item()).type(torch.int64).to(device)], dim=1
        )

        if next_word == eos_idx:
            break

    return decoder_input.squeeze(0)

def run_validation(model, validation_ds, tokenizer_tgt, max_len):
    model.eval()

    source_texts = []
    expected = []
    predicted = []

    with torch.no_grad():
        for i, batch in enumerate(validation_ds):
            print(i, "done")
            model_out = greedy_decode(model, batch["src_text"], batch["encoder_mask"].to(device), tokenizer_tgt, max_len)

            source_text = batch["src_text"][0]
            target_text = batch["tgt_text"][0]
            model_out_text = ""
            for i in model_out.detach().tolist():
                model_out_text += tokenizer_tgt.decode([i])

            source_texts.append(source_text)
            expected.append(target_text)
            predicted.append(model_out_text)
    
    # Calculate character-wise accuracy for each prediction
    def charwise_acc(pred, exp):
        if not exp:
            return 0.0
        # Pad the shorter string so zip works for all chars
        max_len = max(len(pred), len(exp))
        pred = pred.ljust(max_len)
        exp = exp.ljust(max_len)
        correct = sum(p == e for p, e in zip(pred, exp))
        return correct / max_len

    charwise_accs = [charwise_acc(p, e) for p, e in zip(predicted, expected)]

    # Create a DataFrame from the results
    table = pd.DataFrame({
        "input": source_texts,
        "label": expected,
        "predicted": predicted,
        "charwise_accuracy": charwise_accs
    })

    # Save the table to a CSV file
    table.to_csv("../results/eval_samples.csv", index=False)
    return table


config = get_config()
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = build_transformer_esmc(tokenizer_tgt.get_vocab_size(), config['tgt_seq_len'], d_model=config['d_model']).to(device)

# If the user specified a model to preload before training, load it
model_filename = get_weights_file_path(config) if config['preload'] else None
if model_filename:
    print(f'Preloading model {model_filename}')
    state = torch.load(model_filename)
    model.load_state_dict(state['model_state_dict'])
else:
    print('No model to preload, starting from scratch')

df = run_validation(model, val_dataloader, tokenizer_tgt, 100)
print(sum(df["charwise_accuracy"]) / len(df))


/home/pooriya/anaconda3/envs/new_ted/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 7297.61it/s]


Preloading model weights/tmodel__latest.pt
0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
0.26833263251743167


In [2]:
df = pd.read_csv("../results/eval_samples.csv")
print(sum(df["charwise_accuracy"]) / len(df))

0.17634420020401081


In [5]:
df = pd.read_csv("../results/results.csv")
print(sum(df["token_accuracy"]) / len(df))

0.41973127191310583


In [6]:
# label,new_pred
df = pd.read_csv("../results/results_new_2column.csv")
def charwise_acc(pred, exp):
    if not exp:
        return 0.0
    # Pad the shorter string so zip works for all chars
    max_len = max(len(pred), len(exp))
    pred = pred.ljust(max_len)
    exp = exp.ljust(max_len)
    correct = sum(p == e for p, e in zip(pred, exp))
    return correct / max_len

charwise_accs = [charwise_acc(p, e) for p, e in zip(df["new_pred"], df["label"])]

print(sum(charwise_accs) / len(charwise_accs))

0.17876832055200503
